# Biopython notebook tutorial


This basic tutorial shows you how to use Biopython and some of its functions.

## Setup

Imports

In [ ]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython and py3Dmol first
    !pip install biopython
    !pip install py3Dmol
except ImportError:
    pass

In [ ]:
import os
import sys
from urllib.request import urlretrieve
import py3Dmol

import Bio
from Bio import SeqIO, SearchIO, AlignIO, Entrez, Phylo
from Bio.SeqRecord import SeqRecord
from Bio.Blast import NCBIWWW, NCBIXML
import matplotlib.pyplot as plt
from Bio.Seq import Seq
from Bio.SeqUtils import gc_fraction
from Bio.PDB import PDBList
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

Entrez.email = "biofuzje1@gmail.com"

Input file

In [ ]:
input_file = "sequence_nucl.fasta"
fasta_loc = ("https://raw.githubusercontent.com/kn-bibs/biofuzje-workshop/main/unknown_sequence.fasta")


if not os.path.exists(input_file):
    urlretrieve(fasta_loc, input_file)

## Basic properties

In [ ]:
for record in SeqIO.parse(input_file, "fasta"):
    print(record.id)

There is just a single sequence with header "Unknown_sequence". We are not
dealing with many chromosomes, scaffolds or contigs.

Extract the sequence

In [ ]:
record = SeqIO.read(input_file, "fasta")
record.seq

In [ ]:
print("Sequence length (bp)")

The sequence length is 888b, it is pretty small.

In [ ]:
print("GC content", gc_fraction(record))

The GC content is ~0.40, it is within a 'normal' range.

In [ ]:
 record.count("A")

We can count nucleotides and specific motifs. There are 304 adenines in the sequence. Now, let's look for "AGAG" motifs.

There are 9 "AGAG" motifs.

We can easily create the reverse complement sequence.

In [ ]:
record.seq.reverse_complement()

We can also quickly translate the nucleotide sequence to amino acids.

In [ ]:
protein_sequence = record.seq.translate(to_stop=False)
protein_sequence

It is also very easy to check wheather the sequence start with a specific amino acid.

In [ ]:
print("Does the sequence begin with a start codon?\n",
      protein_sequence.startswith("M"))

In [ ]:
print("Protein sequence length in amino acids")

## Comparing to other proteins

Let's use BLAST to align the unknown sequence to other annoated sequences in the NCBI protein database, which contains sequences from many different species. It may take around 5 minutes.

In [ ]:
result_handle = NCBIWWW.qblast("blastp", "nr", protein_sequence, hitlist_size=100)
blast_qresult = SearchIO.read(result_handle, "blast-xml")
print("BLAST hits:", len(blast_qresult.hits))

In [ ]:
'''blast_file = "blast.xml"
blast_loc = ("https://raw.githubusercontent.com/kn-bibs/biofuzje-workshop/main/blast.xml")

if not os.path.exists(blast_file):
  urlretrieve(blast_loc, blast_file)

result_handle = open(blast_file, "rb")
blast_qresult = SearchIO.read(result_handle, "blast-xml")
print("BLAST hits:", len(blast_qresult.hits))'''

Let's look at some of the first hits.

In [ ]:
for hit in blast_qresult.hits[:30]:
  print(hit.id, hit.description)

It looks like our protein comes from Banana bunchy top virus. Let's look closer at the proteins founded by BLAST. First, let's find accesion numbers for our hits!

In [ ]:
accessions = []

for hit in blast_qresult.hits:
    pass


Let's extract a bit more structured meta-data on the top matching sequence homologous sequence using NCBI Entrez via Biopython to extract a GenBank file. Start by separating NCBI_id for the first hit.

In [ ]:
NCBI_id=

NCBI accession from the first hit is:NP_604483.1.

We can look at genbank record for our found protein.

In [ ]:
handle = Entrez.efetch(db="protein", id= NCBI_id, retmode="text", rettype="gb")
genbank_record = SeqIO.read(handle, "genbank")
genbank_record

There's a lot of information in the genbank record if you know where to find it...

In [ ]:
print("What molecule type is it?\n",
      genbank_record.annotations["molecule_type"])

In [ ]:
print("What is the full NCBI taxonomy of the organism that this protein comes from?\n",
      genbank_record.annotations[""])

In [ ]:
print("How many features are there?\n", len(genbank_record.features))

In [ ]:
print("What type of features are there?")
for feature in genbank_record.features:
  print(feature.type)

In [ ]:
print("What are the relevant references/labs who generated the data?\n")
for reference in genbank_record.annotations["references"]:
    print(reference)

Now we can read up more about the protein.

We can also compare found sequences. Let's allign our best hit with our unknown sequence. Can you see a hidden message?

In [ ]:
alignments = pairwise2.align.globalxx(protein_sequence, genbank_record.seq)

for alignment in alignments:
    print(format_alignment(*alignment))

##3D structures

Let's explore our ability to look at pretty proteins. This one is not related to our unknown sequence. It's just pretty.

In [ ]:
pdb_code = "6j5t"

pdb_filename = PDBList().retrieve_pdb_file(pdb_code, file_format="pdb", pdir=".")

with open(pdb_filename, "r") as file:
    pdb_data = file.read()

view = py3Dmol.view(width=800, height=600)
view.addModel(pdb_data, "pdb")
view.setStyle({"cartoon": {"color": "spectrum"}})
view.zoomTo()
view.show()

## Downloading Uniprot records
Sometimes we need Uniprot accession number (for example for accessing alphafold.ebi.ac.uk database), but we only have NCBI ID. Then we can simply search Uniprot with that NCBI ID. To do that in Python we can call Uniprot API.
Exaple URL for API call: https://rest.uniprot.org/uniprotkb/search?query=NP_976067&size=1

In [ ]:
import requests, sys, json

NCBI_id="A0A650G2E0"

params = {
  "query": NCBI_id,
  "size": "1"
}
headers = {
  "accept": "application/json"
}
base_url = "https://rest.uniprot.org/uniprotkb/search"

response = requests.get(base_url, headers=headers, params=params)
if not response.ok:
  response.raise_for_status()
  sys.exit()

data = response.json()

print(json.dumps(data, indent=2)[:1000])

Now let's extract Uniprot accession number

In [ ]:
uniprot_accession = # your code
uniprot_accession

## Downloading structure from Alphafold database
First we get alphafold for this protein (based on Uniprot accession).

In [ ]:
url = f"https://alphafold.ebi.ac.uk/api/prediction/{uniprot_accession}"

try:
    response = requests.get(url)
    if response.status_code == 200:
        alphafold_data = response.json()
        print(alphafold_data)
    else:
        print(f"Error: {response.status_code} - {response.text}")

except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

Now we can download .pdb file

In [ ]:
pdb_url = alphafold_data[0]['pdbUrl']
print(pdb_url)
local_filename = pdb_url.split('/')[-1]
try:
    response = requests.get(pdb_url, stream=True)

    if response.status_code == 200:
        with open(local_filename, "wb") as file:
            file.write(response.content)
        print(f"File downloaded successfully as '{local_filename}'")
    else:
        print(f"Error: {response.status_code} - {response.text}")

except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

Now let's see if our file really contains structure of a protein.

In [ ]:
with open(local_filename, "r") as f:
    pdb_data = f.read()
# using py3dmol show 3D structure of this protein (as previously)

There is tons of other functionality in Biopython, this is just a very small fraction of the modules, see the extensive [official tutorial documentation](http://biopython.org/DIST/docs/tutorial/Tutorial.html). Also, don't hesitate to contact us if you have any more questions: [KNB webside](http://bioinformatyka.mimuw.edu.pl/pl/). Thank you for attending our workshops! We hope you had fun!